In [81]:
import tensorflow as tf
import numpy as np
import pandas as pd
import csv
import json
from sklearn.model_selection import train_test_split
import plotly.express as px
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter
from nltk.corpus import stopwords

In [82]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## 2.1

**Load data:**

In [83]:
df_1 = pd.read_csv('/content/gdrive/My Drive/fashion/All_Beauty.csv', usecols=['overall', 'reviewText'])
df_2 = pd.read_csv('/content/gdrive/My Drive/fashion/AMAZON_FASHION.csv', usecols=['overall', 'reviewText'])
df_3 = pd.read_csv('/content/gdrive/My Drive/fashion/Arts_Crafts_and_Sewing.csv', usecols=['overall', 'reviewText'])
df_4 = pd.read_csv('/content/gdrive/My Drive/fashion/Clothing_Shoes_and_Jewelry_5.csv', usecols=['overall', 'reviewText'])
df_5 = pd.read_csv('/content/gdrive/My Drive/fashion/Sports_and_Outdoors_5.csv', usecols=['overall', 'reviewText'])
df_test = pd.read_csv('/content/gdrive/My Drive/fashion/Luxury_Beauty_5.csv', usecols=['overall', 'reviewText'])

In [85]:
print(f'All Beauty consist of: {len(df_1)} reviews')
print(f'Amazon Fashion consist of: {len(df_2)} reviews')
print(f'Arts Crafts and Sewing consist of: {len(df_3)} reviews')
print(f'Clothing Shoes and Jewelry of: {len(df_4)} reviews')
print(f'Sports and Outdoors consist of: {len(df_5)} reviews')
print(f'Luxury Beauty consist of: {len(df_test)} reviews')

All Beauty consist of: 371345 reviews
Amazon Fashion consist of: 883636 reviews
Arts Crafts and Sewing consist of: 1000000 reviews
Clothing Shoes and Jewelry of: 1000000 reviews
Sports and Outdoors consist of: 1000000 reviews
Luxury Beauty consist of: 34278 reviews


## 2.2

**Exploratory data analysis:**

In [86]:
def copy_and_sample(df, n=10000):
    if len(df) > n:
        return df.sample(n)
    else:
        return df.copy()

df_1_stats = copy_and_sample(df_1)
df_2_stats = copy_and_sample(df_2)
df_3_stats = copy_and_sample(df_3)
df_4_stats = copy_and_sample(df_4)
df_5_stats = copy_and_sample(df_5)

df_1_stats['wordCount'] = df_1['reviewText'].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)
df_2_stats['wordCount'] = df_2['reviewText'].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)
df_3_stats['wordCount'] = df_3['reviewText'].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)
df_4_stats['wordCount'] = df_4['reviewText'].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)
df_5_stats['wordCount'] = df_test['reviewText'].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)

df_1_stats['category'] = 'All_Beauty'
df_2_stats['category'] = 'Amazon_fashion'
df_3_stats['category'] = 'Arts_Crafts_and_Sewing'
df_4_stats['category'] = 'Clothing_Shoes_and_Jewelry'
df_5_stats['category'] = 'Sports_and_Outdoors'

violin_plot_df = pd.concat([df_1_stats, df_2_stats, df_3_stats, df_4_stats, df_5_stats])

fig = px.box(violin_plot_df, x="category", y="wordCount", points="all", color="category")

fig.update_layout(
    title='Word Count Distribution by Category:',
    xaxis=dict(showgrid=True, gridwidth=1, gridcolor='LightGrey'),
    yaxis=dict(showgrid=True, gridwidth=1, gridcolor='LightGrey'),
    plot_bgcolor="#FFFFFF")

fig.update_yaxes(range=[0, 300])

fig.show()

In [100]:
nltk.download('punkt')
nltk.download('stopwords')

def tokenize_and_lowercase(df, column_name):

    stop_words = set(stopwords.words('english'))

    all_words = []
    for text in df[column_name]:
        if isinstance(text, str):
            words = word_tokenize(text.lower())
            all_words.extend([word for word in words if word.isalpha() and word not in stop_words])

    word_counts = Counter(all_words)
    most_common_words = word_counts.most_common(2000)
    return most_common_words

All_Beauty_WORDS = tokenize_and_lowercase(df_1_stats, 'reviewText')
Amazon_fashion_WORDS = tokenize_and_lowercase(df_2_stats, 'reviewText')
Arts_Crafts_and_Sewing_WORDS = tokenize_and_lowercase(df_3_stats, 'reviewText')
Clothing_Shoes_and_Jewelry_WORDS = tokenize_and_lowercase(df_4_stats, 'reviewText')
Sports_and_Outdoors_WORDS = tokenize_and_lowercase(df_5_stats, 'reviewText')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
combined_words = All_Beauty_WORDS + Amazon_fashion_WORDS + Arts_Crafts_and_Sewing_WORDS + Clothing_Shoes_and_Jewelry_WORDS + Sports_and_Outdoors_WORDS

combined_word_counts = Counter()
for word, count in combined_words:
    combined_word_counts[word] += count

top_25_words = combined_word_counts.most_common(15)

top_25_words

In [104]:
def list_to_dict(lst):
    return dict(lst)

All_Beauty_WORDS_dict = list_to_dict(All_Beauty_WORDS)
Amazon_fashion_WORDS_dict = list_to_dict(Amazon_fashion_WORDS)
Arts_Crafts_and_Sewing_WORDS_dict = list_to_dict(Arts_Crafts_and_Sewing_WORDS)
Clothing_Shoes_and_Jewelry_WORDS_dict = list_to_dict(Clothing_Shoes_and_Jewelry_WORDS)
Sports_and_Outdoors_WORDS_dict = list_to_dict(Sports_and_Outdoors_WORDS)

def count_common_keys_percentage(dict_a, dict_b):
    common_keys = len(set(dict_a.keys()) & set(dict_b.keys()))
    total_unique_keys = len(set(dict_a.keys()) | set(dict_b.keys()))
    if total_unique_keys > 0:
        return round((common_keys / total_unique_keys) * 100)
    else:
        return 0

def create_correlation_matrix(dict_list, dict_names):
    matrix = []

    for dict_a in dict_list:
        row = []
        for dict_b in dict_list:
            row.append(count_common_keys_percentage(dict_a, dict_b))
        matrix.append(row)

    df = pd.DataFrame(matrix, index=dict_names, columns=dict_names)
    return df

dicts = [All_Beauty_WORDS_dict, Amazon_fashion_WORDS_dict, Arts_Crafts_and_Sewing_WORDS_dict, Clothing_Shoes_and_Jewelry_WORDS_dict, Sports_and_Outdoors_WORDS_dict]
dict_names = ['All Beauty', 'Amazon Fashion', 'Arts Crafts and Sewing', 'Clothing Shoes and Jewelry', 'Sports and Outdoors']

correlation_matrix = create_correlation_matrix(dicts, dict_names)
correlation_matrix

,All Beauty,Amazon Fashion,Arts Crafts and Sewing,Clothing Shoes and Jewelry,Sports and Outdoors
All Beauty,100,48,51,48,46
Amazon Fashion,48,100,49,65,48
Arts Crafts and Sewing,51,49,100,49,50
Clothing Shoes and Jewelry,48,65,49,100,52
Sports and Outdoors,46,48,50,52,100


In [ ]:
df_train = pd.concat([df_1, df_2, df_3, df_4], axis=0, ignore_index=True, shuffle=True)
df_train = df_train.dropna()
df_test = df_test.dropna()

df_train = df_train[df_train["overall"] != '3']
df_train["label"] = df_train["overall"].apply(lambda rating : 1 if str(rating) > '3' else 0)

df_test = df_test[df_test["overall"] != '3']
df_test["label"] = df_test["overall"].apply(lambda rating : 1 if str(rating) > '3' else 0)

In [ ]:
print(f'First review = {df_train.loc[0, "reviewText"]}')
print(f'First review has length = {len(df_train.loc[0, "reviewText"])}\n ')
print(f'First review overall rating = {df_train.loc[0, "overall"]}')
print(f'First review binary rating = {df_train.loc[0, "label"]}')

First review = As advertised. Reasonably priced
First review has length = 32
 
First review overall rating = 5.0
First review binary rating = 1


**Train:**

In [ ]:
max_tokens = 1000
output_sequence_length = 100
pad_to_max_tokens = True

df_train['reviewText'] = df_train['reviewText'].fillna('').astype(str)

encoder = tf.keras.layers.TextVectorization(max_tokens=max_tokens, output_sequence_length=output_sequence_length, pad_to_max_tokens=pad_to_max_tokens)

text_ds = tf.data.Dataset.from_tensor_slices(df_train['reviewText']).batch(128)
encoder.adapt(text_ds)
vocab = np.array(encoder.get_vocabulary())

train_ds = tf.data.Dataset.from_tensor_slices((df_train['reviewText'], df_train['label'])).batch(128)
train_ds = train_ds.map(lambda x, y: (encoder(x), y))

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)

KeyboardInterrupt: ignored

**Test:**

In [ ]:
df_test['reviewText'] = df_test['reviewText'].fillna('').astype(str)

text_test_ds = tf.data.Dataset.from_tensor_slices(df_test['reviewText']).batch(128)
test_ds = text_test_ds.map(lambda x: encoder(x))

AUTOTUNE = tf.data.AUTOTUNE
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
for text_batch, label_batch in train_ds.take(1):
    for i in range(5):
        print("Review:", text_batch.numpy()[i])
        print("Label:", label_batch.numpy()[i])
        print("---")

Review: [ 10  12   2  90  59   4  58   1   4  99   1   2 883   9  10  31  17 522
  19   1   5   1  10  31  12 241  53   4  12 178  11   3 896   2   1   4
  51 402  10  31  65   3  12  66 334 306   4 200  10  31   3 613  54 385
  96  11  64   3  43   1 319   1   5   1   4  12 178  11   6 104  64   1
  40   4 956  10  31  16  13   2 288   3 534  63   7  39  38 192   4  32
   2 369   1   1   1   1 149 180 107   7]
Label: 1
---
Review: [ 23  19  34   9 684   1 157   4 146  10   8   6 382  58   1   7 123 186
   1   1  52 544   1   1   1   2  27   1   9   2   1   1 134   5   1   1
   9   1   1   5   1   1   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]
Label: 1
---
Review: [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [ ]:
embedding_dimension = 128
embedding_model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=len(vocab),
                              output_dim=embedding_dimension,
                              input_length=100,
                              name="embedding"),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

embedding_model.compile(optimizer='adam',
                        loss='binary_crossentropy',
                        metrics=['accuracy'])

embedding_model.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          128000    
                                                                 
 bidirectional (Bidirection  (None, 100, 256)          263168    
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 100, 256)          0         
                                                                 
 lstm_6 (LSTM)               (None, 64)                82176     
                                                                 
 dense_5 (Dense)             (None, 64)                4160      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                      

In [ ]:
embedding_weights = embedding_model.get_layer('embedding').get_weights()[0]
print(f'Dimension of the embedding vector: \n{embedding_weights.shape}')

Dimension of the embedding vector: 
(1000, 128)


In [ ]:
print(vocab[500:550])

['date' 'chargers' 'car' 'break' 'working' 'wonderful' 'war' 'wants'
 'update' 'top' 'told' 'today' 'show' 'short' 'seems' 'room' 'putting'
 'paint' 'paid' 'opened' 'often' 'multiple' 'live' 'light' 'itself' 'held'
 'forward' 'expect' 'dad' 'assistant' '700' '20' 'young' 'understand'
 'turned' 'thank' 'stop' 'someone' 'silver' 'sienna' 'service' 'reader'
 'quinn' 'point' 'past' 'others' 'ones' 'okay' 'oem' 'novels']


In [ ]:
def get_word_index(word, encoder):
    try:
        return encoder.get_vocabulary().index(word)
    except ValueError:
        return None

words = ['dad', 'today']

for word in words:
    word_index = get_word_index(word, encoder)
    if word_index is not None:
        word_vector = embedding_weights[word_index]
        print(f'{word}: {np.round(word_vector, 3)}')
    else:
        print(f'"{word}" not in vocabulary')


dad: [ 0.015  0.042  0.046  0.022 -0.042 -0.027  0.001 -0.009  0.036  0.003
 -0.012 -0.049 -0.048  0.032 -0.006  0.027  0.003  0.031 -0.021  0.037
 -0.033  0.048 -0.039 -0.011  0.045 -0.003 -0.036 -0.029 -0.023 -0.007
 -0.009 -0.026  0.002 -0.018 -0.047 -0.003 -0.023 -0.006 -0.027 -0.017
  0.046 -0.014 -0.025 -0.049 -0.046 -0.045  0.003  0.017 -0.022  0.05
  0.038 -0.047  0.021 -0.048 -0.033  0.019  0.015  0.037 -0.029 -0.029
 -0.019 -0.003 -0.016 -0.02  -0.02  -0.04   0.049 -0.022 -0.022  0.024
  0.047  0.025  0.017  0.014 -0.021 -0.001  0.01  -0.041  0.002 -0.006
  0.039 -0.022  0.044  0.031 -0.047  0.016 -0.028 -0.001  0.021 -0.001
 -0.037  0.02   0.008 -0.025 -0.012 -0.015 -0.047 -0.049 -0.015  0.017
  0.025  0.006  0.015  0.003  0.003 -0.037 -0.032 -0.035 -0.034 -0.005
  0.041  0.032  0.045  0.018 -0.031  0.011  0.044  0.031  0.025  0.02
  0.035  0.046  0.021  0.033 -0.001  0.044  0.006 -0.006]
today: [-0.025 -0.047 -0.049 -0.032 -0.046  0.017 -0.04  -0.039 -0.039 -0.049
  0.012 -

In [ ]:
embedding_model.compile(optimizer='adam',
                        loss='binary_crossentropy',
                        metrics=['accuracy'])

embedding_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          128000    
                                                                 
 bidirectional (Bidirection  (None, 100, 256)          263168    
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 100, 256)          0         
                                                                 
 lstm_6 (LSTM)               (None, 64)                82176     
                                                                 
 dense_5 (Dense)             (None, 64)                4160      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                      

In [ ]:
embedding_model.fit(train_ds, epochs=10, verbose=1)

Epoch 1/10
 9/16 [===============>..............] - ETA: 6s - loss: 0.3693 - accuracy: 0.8924

KeyboardInterrupt: ignored

In [ ]:
embedding_model.fit(train_ds, epochs=10, verbose=1)
predictions = embedding_model.predict(test_ds)

8/8 [==============================] - 2s 229ms/step


In [ ]:
print(predictions)

[[0.8410684 ]
 [0.629067  ]
 [0.84107214]
 [0.84107214]
 [0.84107214]
 [0.8410719 ]
 [0.84107214]
 [0.84107214]
 [0.84107214]
 [0.84107214]
 [0.84107214]
 [0.841071  ]
 [0.84107196]
 [0.84107214]
 [0.84105265]
 [0.84107214]
 [0.84107214]
 [0.84107214]
 [0.84107214]
 [0.84107155]
 [0.84107214]
 [0.84107214]
 [0.84107214]
 [0.8405561 ]
 [0.84107214]
 [0.84107214]
 [0.84107214]
 [0.84107214]
 [0.8410718 ]
 [0.84107214]
 [0.84107214]
 [0.84107214]
 [0.8409079 ]
 [0.8361913 ]
 [0.8410677 ]
 [0.84107214]
 [0.84107167]
 [0.84107214]
 [0.84107214]
 [0.84107214]
 [0.84107214]
 [0.84107214]
 [0.8410719 ]
 [0.84107053]
 [0.8410721 ]
 [0.84107214]
 [0.84107214]
 [0.84107214]
 [0.84107214]
 [0.84107214]
 [0.84107214]
 [0.8410708 ]
 [0.84107214]
 [0.8410717 ]
 [0.84107214]
 [0.84107196]
 [0.84107214]
 [0.84107214]
 [0.84107214]
 [0.84107214]
 [0.84107214]
 [0.84107214]
 [0.84107214]
 [0.84107214]
 [0.84107214]
 [0.8410686 ]
 [0.8410721 ]
 [0.84107214]
 [0.84107214]
 [0.84107214]
 [0.8410721 ]
 [0.84